In [1]:
#install.packages(c("RDFTensor", "dplyr"))

library(dplyr)
library(RDFTensor)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loading required package: pracma


Attaching package: 'pracma'


The following objects are masked from 'package:Matrix':

    expm, lu, tril, triu


Loading required package: doParallel

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [2]:
#setwd("D://LA//ATI Data")

In [3]:
#Auth <- read.table(file="Summaries//Just_Auth//LA.txt", header=T, sep=",")
#Auth <- Auth[, -1]

In [2]:
Auth <- read.csv(file = 'G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/Advanced-Cyber-Analytics-for-Attack-Detection/Data/Reduced/auth_data_reduced_mk2.csv.gz')
Auth <- Auth[, -1]
colnames(Auth) <- c("UserName","SrcDevice","DstDevice","AuthType", "Failure", "DailyCount")

In [3]:
head(Auth)
dim(Auth)
Auth %>% mutate(across(where(is.factor), as.character)) -> Auth

str(Auth)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
1,User035855,Comp808475,Comp081330,TGS,0,17
2,Comp655251$,Comp655251,ActiveDirectory,NetworkLogon,0,350
3,User384215,Comp095190,EnterpriseAppServer,NetworkLogon,0,35
4,User043263,Comp883307,Comp384394,TGS,0,2
5,User631552,Comp621781,Comp915658,NetworkLogon,0,1
6,User682124,Comp911231,Comp658422,NetworkLogon,0,14


[1] 12391819        6

'data.frame':	12391819 obs. of  6 variables:
 $ UserName  : chr  "User035855" "Comp655251$" "User384215" "User043263" ...
 $ SrcDevice : chr  "Comp808475" "Comp655251" "Comp095190" "Comp883307" ...
 $ DstDevice : chr  "Comp081330" "ActiveDirectory" "EnterpriseAppServer" "Comp384394" ...
 $ AuthType  : chr  "TGS" "NetworkLogon" "NetworkLogon" "TGS" ...
 $ Failure   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ DailyCount: num  17 350 35 2 1 14 2 4 3 6 ...


### Trying out something different

In [4]:
train <- sample(1:nrow(Auth), size=nrow(Auth)*10^-4, replace=FALSE)
U17<- Auth[train, ]

head(U17)
dim(U17)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
402607,Comp645686$,Comp645686,ActiveDirectory,NetworkLogon,0,350
7584833,Comp195549$,Comp195549,Comp253429,TGS,0,1
7006525,Comp349139$,Comp349139,Comp349139,TGS,0,23
7047873,Comp130923$,Comp130923,ActiveDirectory,TGT,0,6
10872896,Comp988334$,Comp988334,ActiveDirectory,TGT,0,1
11436735,User672405,Comp744688,ActiveDirectory,TGS,0,18


[1] 1239    6

In [5]:
t1<- tapply(U17[,1], U17[,1])
t2<- tapply(U17[,2], U17[,2])
t3<- tapply(U17[,3], U17[,3])
t4<- tapply(U17[,4], U17[,4])
t5<- tapply(U17[,5], U17[,5])
t6<- tapply(U17[,6], U17[,6])

T<- as.data.frame(cbind(t1,t2,t3,t4,t5,t6))
head(T)

,t1,t2,t3,t4,t5,t6
,<int>,<int>,<int>,<int>,<int>,<int>
1,342,710,1,4,1,157
2,104,215,44,9,1,1
3,195,398,69,9,1,23
4,70,143,1,10,1,6
5,527,1112,1,10,1,1
6,970,825,1,9,1,18


In [6]:
T<- as.matrix(T)
X=list(subs=T, vals=rep(1, nrow(T)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3)), length(unique(t4)), length(unique(t5)), length(unique(t6))))

print(c(length(unique(t1)), length(unique(t2)), length(unique(t3)), length(unique(t4)), length(unique(t5)), length(unique(t6))))

P4<- cp_apr(X, 2, opts=list(alg='pdnr'))
Pt4 <- as.data.frame(P4$M$u[1])
pt4<- Pt4[order(Pt4[,1], decreasing=TRUE), ]

[1] 1195 1126  205   12    2  210
[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 5052, KKT viol = 1.00e+00, obj = -4.28718570e+05, nz: 1791, time:2.46"
[1] "   2. Ttl Inner Its: 15251, KKT viol = 2.10e+04, obj = -1.27001236e+04, nz: 2476, time:7.92"
[1] "   3. Ttl Inner Its: 2526, KKT viol = 3.83e+00, obj = -1.27001236e+04, nz: 2476, time:8.18"
[1] "   4. Ttl Inner Its: 2665, KKT viol = 3.83e+00, obj = -1.26701683e+04, nz: 2486, time:8.56"
[1] "   5. Ttl Inner Its: 2526, KKT viol = 3.81e-02, obj = -1.26701683e+04, nz: 2486, time:8.86"
[1] "   6. Ttl Inner Its: 6400, KKT viol = 4.14e-02, obj = -1.26924231e+04, nz: 2486, time:10.65"
[1] "   7. Ttl Inner Its: 2554, KKT viol = 3.44e-03, obj = -1.26924231e+04, nz: 2486, time:10.95"
[1] "   8. Ttl Inner Its: 3652, KKT viol = 4.25e-04, obj = -1.26924231e+04, nz: 2486, time:11.67"
[1] "   9. Ttl Inner Its: 2586, KKT viol = 1.28e-04, obj = -

In [7]:
head(pt4)
dim(pt4[pt4$X1>0.00128, ])
R<- as.numeric(rownames(pt4[pt4$X1>0.00128, ]))
R

,X1,X2
,<dbl>,<dbl>
956,0.005805313,0
659,0.002902886,0
1023,0.002902886,0
876,0.002902886,0
136,0.002902885,0
616,0.002902884,0


[1] 665   2

[1]  956  659 1023  876  136  616 1090  692  976 1073  459  978  705 1166 1083
 [16]  557  658  648  695  296  574  228  528  851  516   92  195  418  464  237
 [31] 1154   62    5 1004  525  948 1157  404 1086  922  443  121 1054 1006  736
 [46]  595  544  610  156  628  491  428  324  812  509 1184   64  674  462  804
 [61] 1145  371   47  903 1016  130  381   50 1015  312  775  644  762  822 1186
 [76]  375  597  969  623  524  744    3  466 1020  419  752  684  142  679  570
 [91]  581  906  154  272  149   73  451  763  917  565  196 1057  249  685 1118
[106] 1032  318  677 1149  633  199  257  889  216  999 1001  178 1075  539  122
[121]   75  219  456  379   23  512 1000  110  706 1060  662  868 1066  436  691
[136]  865  387 1127  875   22 1072 1156  329  926  800  778  432  785  366  937
[151]  474  352  151  244  417   39  832  619  499  972  759 1168  182  344  449
[166] 1183  640  190  360  345  530 1135  572 1076  934  857  609  271 1125  470
[181]  523  671  171  266  538 1029  869   41  242  892   60  967 1164  668  489
[196]  227  307  224 1109  357  384  177  347  909 1047  248  135  600  955 1011
[211] 1044 1111  290 1022 1030  541  840 1112  490  336  615 1095  575 1119 1173
[226] 1180  582  328  806  783  202  350  478  973 1084  824 1110   44  749 1121
[241]  754  262 1085  498  165  985  209  398  256  697   45  220 1169  664  601
[256]  901  656  774  372  346  748  400 1005  519  389  617  403 1055  820  238
[271]  412  441 1144  657  300  153  277 1114  118 1150  796  179  152  206  124
[286]   35  809  383  560  173  275  669  269  101  722   55  117  140  788  127
[301] 1097  166  395  621  554  992  134   66  172 1058  714  738  943  711  109
[316]  842  592  222  175 1039  515 1120  880 1108  261  870  660   24  900  147
[331]  653  729 1034  406    9  486  180 1043  467  848  354  877  148 1096  310
[346]  535  731  506  397  716  746   13  500  240  860  514   95  571  758  265
[361]  369 1014  305  566  415  247   59  445  168 1092  132  529  416   68  212
[376] 1101  863 1036  429  831   99  482  455  551  143  650 1163  791   31  521
[391]   27  120 1160   53  938  713  818  606 1070  951  604  226  351  975  537
[406]  116  708   74  723  341  231  837   34  802  681  605  715  476  315   70
[421]   30  497 1146  325  475  205   38   84  501  339  913  316 1147  176  936
[436]   11 1089  230  138  622  646  129  155  766  719 1142 1187  358  614  730
[451] 1153  128 1021  734  484  260  359  702  221 1155  114 1134  186   36  846
[466]  364  733  717  960    7  270  573  507  562  732  304   90  884  844  160
[481]  252  750    8  427  568  663  888  569  910 1148  107   49 1010 1129  635
[496]  883  856   94  781  255 1068  949  299 1107  246  966  113  533   51  447
[511]  495  241  320   54    4   16  409   79  786  294  284  194   32   43   48
[526]  108  115  119  211  234  254  285  291  309  317  327  333  353  356  377
[541]  402  414  439  440  454  477  485  503  576  584  587  603  626  630  642
[556]  651  667  678  701  703  742  747  757  773  779  795  813  814  821  825
[571]  859  890  899  915  919  932  986  991 1028 1035 1053 1059 1100 1104 1105
[586] 1113 1137 1158 1161  189 1049 1079  793  994  924  424 1050 1115 1007  624
[601]  935 1130  808  361 1116  163  928  468  123  611 1143  745  688   80 1048
[616]  422  953   67  543  613  843   83  827  399  131  450  737  683  106  835
[631]   14   85  187  784  849  373   52  393  726  761   17  338  591  893  203
[646]  408  433  452  471  520 1091  550  204  771  905 1063  365  435  362 1167
[661]  385   76   57  993  805

In [8]:
T<- as.data.frame(T)
data<- cbind(U17, T)
data<- data[data$t1%in%R, ]

head(data)
dim(data)
L<- unique(data$UserName)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount,t1,t2,t3,t4,t5,t6
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
7006525,Comp349139$,Comp349139,Comp349139,TGS,0,23,195,398,69,9,1,23
7047873,Comp130923$,Comp130923,ActiveDirectory,TGT,0,6,70,143,1,10,1,6
3338371,Comp081299$,Comp081299,Comp908480,TGS,0,1,41,88,177,9,1,1
563722,Comp821880$,Comp821880,Comp939275,TGS,0,1,440,914,188,9,1,1
2446408,User834840,Comp247374,ActiveDirectory,TGT,0,22,1079,286,1,10,1,22
3433735,User718322,Comp490834,Comp004340,TGS,0,1,999,544,3,9,1,1


[1] 690  12

In [9]:
#setwd("D://LA//ATI Data")
AU <- read.table(file="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/Advanced-Cyber-Analytics-for-Attack-Detection/Data/AuthUserNames.txt", header=F)
AU <- as.vector(AU$V1)
AU <- unique(AU)

Y<- L[L%in%AU==TRUE]

print(paste(length(Y)/length(L)*100, "% of the list of usernames detected features in the AU list."))

[1] "0.451127819548872 % of the list of usernames detected features in the AU list."


### User179746

In [10]:
U17 <- Auth[Auth$UserName=='User179746', ]

In [11]:
df<- as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Fail=U17$Failure))

In [12]:
head(df)
dim(df)

,Src,Dst,Fail
,<chr>,<chr>,<chr>
1,Comp599155,None,0
2,Comp599155,None,0
3,Comp599155,None,0
4,Comp679250,Comp599155,0
5,Comp547871,ActiveDirectory,1
6,Comp679250,Comp599155,0


[1] 57  3

In [23]:
SrcT <- tapply(df$Src, df$Src)
DstT <- tapply(df$Dst, df$Dst)
FailT <- tapply(df$Fail, df$Fail)

dm <- as.matrix(cbind(SrcT, DstT, FailT))

In [24]:
head(dm)

head(df)

SrcT,DstT,FailT
3,25,1
3,25,1
3,25,1
4,14,1
2,1,2
4,14,1


,Src,Dst,Fail
,<chr>,<chr>,<chr>
1,Comp599155,None,0
2,Comp599155,None,0
3,Comp599155,None,0
4,Comp679250,Comp599155,0
5,Comp547871,ActiveDirectory,1
6,Comp679250,Comp599155,0


[1] "integer"

In [15]:
subs=matrix(c(5,1,1,
              3,1,2,
              1,1,3,
              2,1,3,
              4,1,3,
              6,1,3,
              1,1,4,
              2,1,4,
              4,1,4,
              6,1,4,
              1,2,1,
              3,2,1,
              5,2,1),byrow=TRUE,ncol=3)
X=list(subs=subs,vals=rep(1,nrow(subs)),size=c(6,2,4))

P1=cp_apr(X,2,opts=list(alg='mu'))
print(P1$M)

[1] "CP_APR: mu"
[1] " Iter    1: Inner Its = 26 KKT violation = 8.596320e-02, nViolations =  0"
[1] " Iter    2: Inner Its =  5 KKT violation = 8.799004e-06, nViolations =  0"
[1] " Iter    3: Inner Its =  3 KKT violation = 8.799004e-06, nViolations =  0"
[1] "Exiting because all subproblems reached KKT tol."
[1] "==========================================="
[1] " Final log-likelihood = -1.609449e+01 "
[1] " Final least squares fit = 6.297194e-01 "
[1] " Final KKT violation = 8.7990037e-06"
[1] " Total inner iterations = 34"
[1] " Total execution time = 0.06 secs"
$lambda
[1] 5.000001 7.999999

$u
$u[[1]]
             [,1]         [,2]
[1,] 2.000002e-01 2.499999e-01
[2,] 8.561663e-10 2.500000e-01
[3,] 3.999999e-01 1.685624e-13
[4,] 2.260438e-09 2.500000e-01
[5,] 3.999999e-01 7.763428e-15
[6,] 4.690343e-10 2.500000e-01

$u[[2]]
          [,1]         [,2]
[1,] 0.4000001 1.000000e+00
[2,] 0.5999999 7.090325e-18

$u[[3]]
             [,1]         [,2]
[1,] 7.999980e-01 1.572617e-12
[2,] 

In [26]:
M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(SrcT)), length(unique(DstT)), 2))
invisible(M)

tfm<- getTensor(trp=dm)

[1] "The modes:"
[1]  4  4 25


In [27]:
set.seed(5)
P1 <- cp_apr(M,2,opts=list(alg='mu'))

print(P1$M)

[1] "CP_APR: mu"
[1] " Iter    1: Inner Its = 23 KKT violation = 1.335418e-01, nViolations =  0"
[1] " Iter    2: Inner Its =  8 KKT violation = 6.241433e-07, nViolations =  0"
[1] " Iter    3: Inner Its =  3 KKT violation = 6.241433e-07, nViolations =  0"
[1] "Exiting because all subproblems reached KKT tol."


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = -2.925011e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 6.2414330e-07"
[1] " Total inner iterations = 34"
[1] " Total execution time = 0.09 secs"
$lambda
[1] 48  9

$u
$u[[1]]
              [,1]         [,2]
[1,] 2.600441e-128 1.111111e-01
[2,]  6.250000e-02 3.213536e-09
[3,]  9.375000e-01 6.934926e-08
[4,] 9.044691e-118 8.888888e-01

$u[[2]]
               [,1]         [,2]
 [1,]  4.166667e-02 3.470756e-34
 [2,]  4.166667e-02 2.555932e-14
 [3,]  2.083333e-02 1.272722e-14
 [4,]  2.083333e-02 6.725709e-15
 [5,]  4.166667e-02 3.273877e-14
 [6,]  2.083333e-02 1.162181e-14
 [7,]  2.083333e-02 6.695594e-15
 [8,]  4.166667e-02 1.690307e-14
 [9,]  2.083333e-02 1.952144e-14
[10,]  2.083333e-02 1.602380e-14
[11,]  2.083333e-02 3.523004e-14
[12,]  4.166667e-02 4.726771e-15
[13,] 3.393511e-156 2.222222e-01
[14,] 9.979302e-156 7.777778e-01
[15,]  4.166667e-02 1.621977e-14
[16,]  1.250000e-01 4.

In [19]:
P1_2<- cp_apr(M,2,opts=list(alg='pdnr'))

print(P1_2$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"


ERROR: Error in M$u[[nn]][X$subs[sparse_indices, nn], ]: no 'dimnames' attribute for array


In [28]:
P1_3<- cp_apr(M,2,opts=list(alg='pqnr'))

print(P1_3$M)

[1] "CP_PQNR (alternating Poisson regression using quasi-Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "-lno=1-i=1--jj:1----tmp_rho:999.021977,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.994799"
[1] "-lno=2-i=2--jj:1----tmp_rho:0.392664,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.880425"
[1] "-lno=3-i=3--jj:1----tmp_rho:0.172231,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.082158"
[1] "-lno=4-i=4--jj:1----tmp_rho:133.342020,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.000843"
[1] "-lno=5-i=1--jj:2----tmp_rho:192.700703,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.947696"
[1] "-lno=6-i=2--jj:2----tmp_rho:0.045380,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.521308"
[1] "-lno=7-i=3--jj:2----tmp_rho:0.115686,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:2.441732"
[1] "-lno=8-i=4--jj:2----tmp_rho:0.091461,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.369811"
[1] "-lno=9-i=5--jj:2----tmp_rho:1.389784,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.023510"
[1] "-lno=10-i=6--jj:2----tmp_rho:315.687157,lbfgsPos:3,

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = -2.925010e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 7.6483841e-05"
[1] " Total inner iterations = 241"
[1] " Total execution time = 0.11 secs"
$lambda
[1] 47.998642  8.999916

$u
$u[[1]]
           [,1]      [,2]
[1,] 0.00000000 0.1111121
[2,] 0.06250403 0.0000000
[3,] 0.93749597 0.0000000
[4,] 0.00000000 0.8888879

$u[[2]]
            [,1]      [,2]
 [1,] 0.04166742 0.0000000
 [2,] 0.04166724 0.0000000
 [3,] 0.02083393 0.0000000
 [4,] 0.02083393 0.0000000
 [5,] 0.04166722 0.0000000
 [6,] 0.02083393 0.0000000
 [7,] 0.02083393 0.0000000
 [8,] 0.04166695 0.0000000
 [9,] 0.02083393 0.0000000
[10,] 0.02083393 0.0000000
[11,] 0.02083393 0.0000000
[12,] 0.04166726 0.0000000
[13,] 0.00000000 0.2222289
[14,] 0.00000000 0.7777711
[15,] 0.04166700 0.0000000
[16,] 0.12499815 0.0000000
[17,] 0.02083393 0.0000000
[18,] 0.04166762 0.0000000
[19,] 0.04166660 0.0000000
[20,] 0.04166633 0.000000

In [29]:
P1_4<- cp_apr(M,2,opts=list(alg='kappatol'))

print(P1_4$M)

            ## The alg is available in the doc but not yet supported as of current R -v

ERROR: Error in cp_apr(M, 2, opts = list(alg = "kappatol")): invalid alg:kappatol, possible values are mu,pdnr,pqnr


**CP_NMU implementation [ Fail ]**

In [30]:
set.seed(15)

P2 <- cp_nmu(M, R=2, opts=list(alg='pdnr'))

[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"


[1] " Iter  1: fit = NaN fitdelta =     NaN, itime=0.00"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"


[1] " Iter  2: fit = NaN fitdelta =     NaN, itime=0.01"


ERROR: Error in if ((iter > 1) && (fitchange < fitchangetol)) {: missing value where TRUE/FALSE needed


In [31]:
tnsr=getTensor(dm)

[1] "The modes:"
[1]  4  4 25


**CP_01 implementation [ Fail ]**

In [32]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

head(df, 10)
class(df) 

Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


[1] "matrix" "array"

In [33]:
tnsr=getTensor(df)
subs=getTnsrijk(tnsr$X)
T<- list(subs=subs,vals=rep(1,nrow(subs)),size=c(length(unique(df[,1])), length(unique(df[,2])), length(unique(df[,3]))))
normT=sqrt(sum(T$vals))

set.seed(555)
P1<- cp_nmu(T, R=2)
res<- CP_R01(T, P1[[1]])

[1] "The modes:"
[1]  7  7 25
[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


ERROR: Error in pracma::accumarray(newsubs, newvals, sz = newsiz): Argument 'sz' does not fit with 'subs'.


##### Same experiment on AuthType

In [34]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

t1<- tapply(df[,1], df[,1])
t2<- tapply(df[,2], df[,2])
t3<- tapply(df[,3], df[,3])

dt<- as.matrix(cbind(t1,t2,t3))

head(dt, 10)
head(df, 10)

t1,t2,t3
3,25,1
3,25,1
3,25,1
4,14,2
2,1,2
4,14,3
4,14,2
1,14,2
3,21,2
3,10,2


Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


In [35]:
MT <- list(subs=dt, vals=rep(1, nrow(dt)), size=c(length(unique(dt[,1])), length(unique(dt[,2])), length(unique(dt[,3]))))

set.seed(25)
P<- cp_apr(MT,2,opts=list(alg='pdnr'))

print(P$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 64, KKT viol = 6.37e+00, obj = -5.31177865e+01, nz: 8, time:0.03"
[1] "   2. Ttl Inner Its: 87, KKT viol = 2.74e+00, obj = -3.18523726e+01, nz: 31, time:0.04"
[1] "   3. Ttl Inner Its: 36, KKT viol = 4.70e-02, obj = -3.18421374e+01, nz: 31, time:0.06"
[1] "   4. Ttl Inner Its: 63, KKT viol = 1.74e-02, obj = -3.18392438e+01, nz: 31, time:0.07"
[1] "   5. Ttl Inner Its: 34, KKT viol = 2.85e-04, obj = -3.18392436e+01, nz: 31, time:0.07"
[1] "   6. Ttl Inner Its: 33, KKT viol = 1.03e-04, obj = -3.18392436e+01, nz: 31, time:0.07"
[1] "   7. Ttl Inner Its: 32, KKT viol = 8.05e-05, obj = -3.18392436e+01, nz: 31, time:0.09"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = -3.183924e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 8.0483328e-05"
[1] " Total inner iterations = 349"
[1] " Total execution time = 0.09 secs"
$lambda
[1]  8.999967 48.001164

$u
$u[[1]]
          [,1]      [,2]
[1,] 0.1111075 0.0000000
[2,] 0.0000000 0.0624941
[3,] 0.0000000 0.9375059
[4,] 0.8888925 0.0000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166843
 [2,] 0.0000000 0.04166814
 [3,] 0.0000000 0.02083244
 [4,] 0.0000000 0.02083414
 [5,] 0.0000000 0.04166431
 [6,] 0.0000000 0.02083115
 [7,] 0.0000000 0.02083307
 [8,] 0.0000000 0.04166389
 [9,] 0.0000000 0.02083422
[10,] 0.0000000 0.02083280
[11,] 0.0000000 0.02083393
[12,] 0.0000000 0.04166565
[13,] 0.2222217 0.00000000
[14,] 0.7777783 0.00000000
[15,] 0.0000000 0.04166781
[16,] 0.0000000 0.12499697
[17,] 0.0000000 0.02083316
[18,] 0.0000000 0.04166484
[19,] 0.0000000 0.04166661
[20,] 0.0000000 0.04166838
[21

In [36]:
P2<- as.data.frame(P$M$u[2])
P2
dim(P2)

X1,X2
<dbl>,<dbl>
0.0000000,0.04166843
0.0000000,0.04166814
0.0000000,0.02083244
0.0000000,0.02083414
0.0000000,0.04166431
0.0000000,0.02083115
0.0000000,0.02083307
0.0000000,0.04166389
0.0000000,0.02083422


[1] 25  2

### User794682

In [37]:
U79 <- Auth[Auth$UserName=='User794682', ]
df<- as.data.frame(cbind(Src=U79$SrcDevice, Dst=U79$DstDevice, Fail=U79$Failure))

t1<- tapply(df$Src, df$Src)
t2<- tapply(df$Dst, df$Dst)
t3<- tapply(df$Fail, df$Fail)

dm<- as.data.frame(cbind(t1,t2,t3))
data<- as.data.frame(cbind(U79, samp=c(1:nrow(df))))
dm<- as.matrix(dm)

head(data, 10)
head(dm, 10)

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount,samp
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<int>
598934,User794682,Comp679250,Comp679250,TGS,0,11,1
599939,User794682,Comp679250,ActiveDirectory,TGS,0,3,2
622421,User794682,Comp679250,None,InteractiveLogon,0,20,3
628371,User794682,Comp679250,None,CachedInteractive,1,10,4
654240,User794682,Comp679250,ActiveDirectory,NetworkLogon,0,1,5
979597,User794682,Comp679250,Comp056180,TGS,0,1,6
1001549,User794682,Comp679250,None,InteractiveLogon,1,1,7
1085196,User794682,Comp679250,None,InteractiveLogon,0,10,8
1089349,User794682,Comp679250,Comp056180,NetworkLogon,0,1,9


t1,t2,t3
3,7,1
3,1,1
3,9,1
3,9,2
3,1,1
3,3,1
3,9,2
3,9,1
3,3,1
3,9,2


In [38]:
M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3))))
invisible(M)

set.seed(5)
P1 <- cp_apr(M,R=2,opts=list(alg='pdnr'))

print(P1$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 28, KKT viol = 3.34e+01, obj = 1.53939704e+02, nz: 5, time:0.02"
[1] "   2. Ttl Inner Its: 33, KKT viol = 2.22e+00, obj = 2.43116298e+02, nz: 8, time:0.03"
[1] "   3. Ttl Inner Its: 42, KKT viol = 4.52e-01, obj = 2.47195151e+02, nz: 9, time:0.05"
[1] "   4. Ttl Inner Its: 32, KKT viol = 4.75e-01, obj = 2.47474419e+02, nz: 9, time:0.06"
[1] "   5. Ttl Inner Its: 27, KKT viol = 3.80e-01, obj = 2.47538950e+02, nz: 10, time:0.06"
[1] "   6. Ttl Inner Its: 16, KKT viol = 5.58e-02, obj = 2.47540922e+02, nz: 10, time:0.08"
[1] "   7. Ttl Inner Its: 21, KKT viol = 1.03e-02, obj = 2.47541138e+02, nz: 10, time:0.08"
[1] "   8. Ttl Inner Its: 21, KKT viol = 1.55e-03, obj = 2.47541144e+02, nz: 10, time:0.09"
[1] "   9. Ttl Inner Its: 18, KKT viol = 8.68e-04, obj = 2.47541144e+02, nz: 10, time:0.09"
[1] "  10. Ttl Inner Its: 14, KKT viol = 7.30e-05,

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = 2.475411e+02 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 7.3006114e-05"
[1] " Total inner iterations = 252"
[1] " Total execution time = 0.09 secs"
$lambda
[1] 56.85751 75.14379

$u
$u[[1]]
           [,1]       [,2]
[1,] 0.01244344 0.08374355
[2,] 0.00000000 0.01330847
[3,] 0.98755656 0.90294798

$u[[2]]
           [,1]       [,2]
 [1,] 0.2788879 0.02852195
 [2,] 0.0703524 0.00000000
 [3,] 0.0703524 0.00000000
 [4,] 0.0000000 0.01330835
 [5,] 0.0703524 0.00000000
 [6,] 0.0000000 0.01330868
 [7,] 0.4924668 0.00000000
 [8,] 0.0175881 0.00000000
 [9,] 0.0000000 0.94486102

$u[[3]]
     [,1]      [,2]
[1,]    1 0.4543795
[2,]    0 0.5456205




In [42]:
Pt1 <- as.data.frame(P1$M$u[1])
Pt2 <- as.data.frame(P1$M$u[2])

s1<- sd(Pt1[,1]) + mean(Pt1[,1])
s2<- sd(Pt2[,1]) + mean(Pt2[,1])

print(c(s1, s2))

[1] 0.8999414 0.2785733


In [43]:
s<- IQR(Pt2[,1])
s1<- sd(Pt2[,1])
s
s1

[1] 0.0703524

[1] 0.1674622

In [44]:
R1<- as.numeric(row.names(Pt1[Pt1$X1>s1, ]))
R2<- as.numeric(row.names(Pt2[Pt2$X1>s2, ]))
R1
R2

[1] 3

[1] 1 7

In [45]:
dm<- as.data.frame(dm)
dmt <- dm[dm$t1%in%R1, ]
dmt <- dmt[dmt$t2%in%R2, ]
dmt <- dmt[dmt$t3==2, ]
dmt

,t1,t2,t3
,<int>,<int>,<int>
114,3,1,2


In [48]:
nrow(dmt)/nrow(U79)*100

[1] 0.7575758

In [49]:
N1<- as.numeric(row.names(dmt))
head(N1)
length(N1)

[1] 114

[1] 1

In [50]:
U79_anom <- data[data$samp%in%N1, ]
U79_anom <- U79_anom[, -ncol(U79_anom)]
head(U79_anom, 20)
identical(nrow(U79_anom), length(N1))

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
10543157,User794682,Comp679250,ActiveDirectory,NetworkLogon,1,3


[1] TRUE

In [51]:
DetectAnomalies <- function(x='User794682', s=1, q=0, m=1) {
    User <- Auth[Auth$UserName==x, ]
    n<- readline(prompt="Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F : ")
    if(n=='F') {
        df<- as.data.frame(cbind(Src=User$SrcDevice, Dst=User$DstDevice, Obj=User$Failure)) }
    if(n=='A') {
        df<- as.data.frame(cbind(Src=User$SrcDevice, Dst=User$DstDevice, Obj=User$AuthType)) }
    if(n!='A' & n!='F') {
        print("Unrecognized input.")
        cat("\n")
        stop()
    }
    
    t1<- tapply(df$Src, df$Src)
    t2<- tapply(df$Dst, df$Dst)
    t3<- tapply(df$Obj, df$Obj)

    dm<- as.data.frame(cbind(t1,t2,t3))
    data<- as.data.frame(cbind(User, samp=c(1:nrow(df))))
    dm<- as.matrix(dm)
    
    set.seed(5)
    M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(t1)), length(unique(t2)), length(unique(t3))))
    invisible(M)
    P1 <- cp_apr(M,2,opts=list(alg='pdnr'))
    cat("\n")
    print(P1$M)
    cat("\n")
    
    Pt1 <- as.data.frame(P1$M$u[1])
    Pt2 <- as.data.frame(P1$M$u[2])
    Pt3 <- as.data.frame(P1$M$u[3])
    p1<- s*sd(Pt1[,1]) + m*mean(Pt1[,1]) + q*IQR(Pt1[,1])
    p2<- s*sd(Pt2[,1]) + m*mean(Pt2[,1]) + q*IQR(Pt2[,1])
    p3<- s*sd(Pt3[,1]) + m*mean(Pt3[,1]) + q*IQR(Pt3[,1])

    R1<- as.numeric(row.names(Pt1[Pt1$X1>p1, ]))
    R2<- as.numeric(row.names(Pt2[Pt2$X1>p2, ]))
    R3<- as.numeric(row.names(Pt3[Pt3$X1>p3, ]))
    
    
    dm<- as.data.frame(dm)
    dmt <- dm[dm$t1%in%R1, ]
    dmt <- dmt[dmt$t2%in%R2, ]
    dmt <- dmt[dmt$t3%in%R3, ]
    
    print(paste("The amount of anomalous data is", nrow(dmt)/nrow(User)*100, "% of the total data provided." ))
    cat("\n")
    
    N1<- as.numeric(row.names(dmt))
    User_anom <- data[data$samp%in%N1, ]
    User_anom <- User_anom[, -ncol(User_anom)]
    print(paste("Verdict for the integrity of the process is:", identical(nrow(User_anom), length(N1))))
    cat("\n\n")
    print("The resulting dataset is:")
    cat("\n")
    head(User_anom, round(nrow(User_anom)/10))
}

In [55]:
DetectAnomalies(s=0, m=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  F


[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 28, KKT viol = 3.34e+01, obj = 1.53939704e+02, nz: 5, time:0.01"
[1] "   2. Ttl Inner Its: 33, KKT viol = 2.22e+00, obj = 2.43116298e+02, nz: 8, time:0.03"
[1] "   3. Ttl Inner Its: 42, KKT viol = 4.52e-01, obj = 2.47195151e+02, nz: 9, time:0.03"
[1] "   4. Ttl Inner Its: 32, KKT viol = 4.75e-01, obj = 2.47474419e+02, nz: 9, time:0.04"
[1] "   5. Ttl Inner Its: 27, KKT viol = 3.80e-01, obj = 2.47538950e+02, nz: 10, time:0.04"
[1] "   6. Ttl Inner Its: 16, KKT viol = 5.58e-02, obj = 2.47540922e+02, nz: 10, time:0.04"
[1] "   7. Ttl Inner Its: 21, KKT viol = 1.03e-02, obj = 2.47541138e+02, nz: 10, time:0.06"
[1] "   8. Ttl Inner Its: 21, KKT viol = 1.55e-03, obj = 2.47541144e+02, nz: 10, time:0.06"
[1] "   9. Ttl Inner Its: 18, KKT viol = 8.68e-04, obj = 2.47541144e+02, nz: 10, time:0.06"
[1] "  10. Ttl Inner Its: 14, KKT viol = 7.30e-05,

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = 2.475411e+02 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 7.3006114e-05"
[1] " Total inner iterations = 252"
[1] " Total execution time = 0.06 secs"

$lambda
[1] 56.85751 75.14379

$u
$u[[1]]
           [,1]       [,2]
[1,] 0.01244344 0.08374355
[2,] 0.00000000 0.01330847
[3,] 0.98755656 0.90294798

$u[[2]]
           [,1]       [,2]
 [1,] 0.2788879 0.02852195
 [2,] 0.0703524 0.00000000
 [3,] 0.0703524 0.00000000
 [4,] 0.0000000 0.01330835
 [5,] 0.0703524 0.00000000
 [6,] 0.0000000 0.01330868
 [7,] 0.4924668 0.00000000
 [8,] 0.0175881 0.00000000
 [9,] 0.0000000 0.94486102

$u[[3]]
     [,1]      [,2]
[1,]    1 0.4543795
[2,]    0 0.5456205



[1] "The amount of anomalous data is 33.3333333333333 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
598934,User794682,Comp679250,Comp679250,TGS,0,11
599939,User794682,Comp679250,ActiveDirectory,TGS,0,3
654240,User794682,Comp679250,ActiveDirectory,NetworkLogon,0,1
1118216,User794682,Comp679250,Comp679250,TGS,0,6


In [56]:
DetectAnomalies("AppService", s=1,m=0,q=0)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  A


ERROR: Error in data.frame(..., check.names = FALSE): arguments imply differing number of rows: 0, 2


To estimate FPR:
Both for my CP_APR and for AE (assuming it too was uniformly applied on the whole 90 days) we look at how many anomalies the models detect Days 1 - 56, since we know there aren't supposed to be any in that time period. 


##### Testing the interaction with non-malicious usernames:  (Failure based approach)

1. **Comp599155$** [Success]

In [57]:
DetectAnomalies("Comp599155$", m=0, s=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  A


[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 366, KKT viol = 5.78e+01, obj = 2.33421880e+02, nz: 7, time:0.19"
[1] "   2. Ttl Inner Its: 240, KKT viol = 8.01e+00, obj = 5.68723129e+02, nz: 5, time:0.23"
[1] "   3. Ttl Inner Its: 506, KKT viol = 1.11e+00, obj = 6.48451374e+02, nz: 3, time:0.42"
[1] "   4. Ttl Inner Its: 384, KKT viol = 2.26e-01, obj = 6.54848842e+02, nz: 33, time:0.53"
[1] "   5. Ttl Inner Its: 389, KKT viol = 1.90e-01, obj = 6.59057582e+02, nz: 41, time:0.64"
[1] "   6. Ttl Inner Its: 412, KKT viol = 2.15e-01, obj = 6.63924069e+02, nz: 42, time:0.73"
[1] "   7. Ttl Inner Its: 459, KKT viol = 1.20e-01, obj = 6.64860215e+02, nz: 97, time:0.86"
[1] "   8. Ttl Inner Its: 184, KKT viol = 1.59e-03, obj = 6.64860222e+02, nz: 97, time:0.88"
[1] "   9. Ttl Inner Its: 246, KKT viol = 7.25e-04, obj = 6.64860230e+02, nz: 97, time:0.94"
[1] "  10. Ttl Inner Its: 183, KKT viol 

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = 6.648602e+02 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.7924271e-05"
[1] " Total inner iterations = 3369"
[1] " Total execution time = 0.95 secs"

$lambda
[1] 309.0010 318.9919

$u
$u[[1]]
     [,1] [,2]
[1,]    1    1

$u[[2]]
              [,1]        [,2]
  [1,] 0.462801362 0.222578820
  [2,] 0.006472048 0.006269140
  [3,] 0.003236306 0.000000000
  [4,] 0.003236024 0.003134572
  [5,] 0.003236024 0.003134572
  [6,] 0.003236024 0.003134572
  [7,] 0.003236024 0.003134572
  [8,] 0.000000000 0.003134876
  [9,] 0.000000000 0.003134876
 [10,] 0.003236306 0.000000000
 [11,] 0.003236024 0.003134572
 [12,] 0.000000000 0.003134876
 [13,] 0.003236024 0.003134571
 [14,] 0.000000000 0.003134876
 [15,] 0.000000000 0.003134876
 [16,] 0.003236024 0.003134572
 [17,] 0.003236306 0.000000000
 [18,] 0.000000000 0.003134876
 [19,] 0.000000000 0.003134876
 [20,] 0.003236024 0.003134572
 [21,] 0.003236

Warning message in DetectAnomalies("Comp599155$", m = 0, s = 0, q = 1):
"NAs introduced by coercion"


[1] "The amount of anomalous data is 0 % of the total data provided."

[1] "Verdict for the integrity of the process is: TRUE"


[1] "The resulting dataset is:"



UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>


2. **User035855** [Success]	

In [58]:
DetectAnomalies("User035855", m=1, s=0, q=1)

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  A


[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 70, KKT viol = 1.81e+01, obj = 1.74700829e+03, nz: 15, time:0.03"
[1] "   2. Ttl Inner Its: 104, KKT viol = 9.69e+01, obj = 2.43052124e+03, nz: 21, time:0.04"
[1] "   3. Ttl Inner Its: 44, KKT viol = 1.88e+00, obj = 2.44057193e+03, nz: 23, time:0.06"
[1] "   4. Ttl Inner Its: 104, KKT viol = 2.19e+00, obj = 2.48452071e+03, nz: 32, time:0.11"
[1] "   5. Ttl Inner Its: 35, KKT viol = 1.81e-02, obj = 2.48452071e+03, nz: 32, time:0.11"
[1] "   6. Ttl Inner Its: 71, KKT viol = 1.81e-02, obj = 2.48455183e+03, nz: 32, time:0.12"
[1] "   7. Ttl Inner Its: 35, KKT viol = 1.32e-04, obj = 2.48455183e+03, nz: 32, time:0.14"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = 2.484552e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 1.3197837e-04"
[1] " Total inner iterations = 463"
[1] " Total execution time = 0.14 secs"

$lambda
[1] 195.9980 870.0042

$u
$u[[1]]
           [,1]        [,2]
[1,] 0.01530625 0.019540663
[2,] 0.00000000 0.001149449
[3,] 0.96938750 0.947127240
[4,] 0.01530625 0.000000000
[5,] 0.00000000 0.032182648

$u[[2]]
            [,1]        [,2]
 [1,] 0.00000000 0.226445846
 [2,] 0.00000000 0.126424095
 [3,] 0.00000000 0.003448363
 [4,] 0.00000000 0.103450532
 [5,] 0.00000000 0.064359010
 [6,] 0.00000000 0.042529314
 [7,] 0.00000000 0.085058451
 [8,] 0.00000000 0.001149462
 [9,] 0.00000000 0.027586894
[10,] 0.00000000 0.001149455
[11,] 0.00000000 0.055174676
[12,] 0.00000000 0.005747219
[13,] 0.00000000 0.001149468
[14,] 0.00000000 0.006896667
[15,] 0.00000000 0.065518495
[16,] 0.00000000 0.064369035
[17,] 0.00000000 0.004597850
[18,] 0

UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>


3. **User762066** [Success]

In [60]:
DetectAnomalies("User762066")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  F


ERROR: Error in data.frame(..., check.names = FALSE): arguments imply differing number of rows: 0, 2


4. **Scanner** [Success]

In [61]:
DetectAnomalies("Scanner")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  A


ERROR: Error in data.frame(..., check.names = FALSE): arguments imply differing number of rows: 0, 2


5. **User631552** [Success]

In [62]:
DetectAnomalies("User631552")

Which feature would you like to consider as object, AuthType [A] or Failure[F]. Input A/F :  A


[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 62, KKT viol = 1.66e+01, obj = 2.09564217e+03, nz: 17, time:0.03"
[1] "   2. Ttl Inner Its: 91, KKT viol = 6.83e+00, obj = 2.70881655e+03, nz: 22, time:0.06"
[1] "   3. Ttl Inner Its: 49, KKT viol = 7.46e-01, obj = 2.73799155e+03, nz: 25, time:0.07"
[1] "   4. Ttl Inner Its: 76, KKT viol = 1.00e+00, obj = 2.79346882e+03, nz: 27, time:0.11"
[1] "   5. Ttl Inner Its: 31, KKT viol = 4.63e-03, obj = 2.79346882e+03, nz: 27, time:0.12"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = 2.793469e+03 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 4.6310828e-03"
[1] " Total inner iterations = 309"
[1] " Total execution time = 0.12 secs"

$lambda
[1] 881.2639 239.3667

$u
$u[[1]]
           [,1]        [,2]
[1,] 0.02154194 0.004166231
[2,] 0.93083974 0.979172945
[3,] 0.00000000 0.004163622
[4,] 0.01020337 0.000000000
[5,] 0.03741494 0.012497203

$u[[2]]
             [,1]        [,2]
 [1,] 0.280154347 0.000000000
 [2,] 0.004523443 0.000000000
 [3,] 0.002270681 0.000000000
 [4,] 0.001135417 0.000000000
 [5,] 0.104272171 0.000000000
 [6,] 0.043146037 0.000000000
 [7,] 0.083793667 0.000000000
 [8,] 0.056773874 0.004168627
 [9,] 0.110839165 0.000000000
[10,] 0.088475159 0.000000000
[11,] 0.010185035 0.000000000
[12,] 0.162358453 0.000000000
[13,] 0.037319361 0.000000000
[14,] 0.001135342 0.000000000
[15,] 0.001131488 0.000000000
[16,] 0.012486360 0.000000000
[17,] 0.000000000 0

,UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
5,User631552,Comp621781,Comp915658,NetworkLogon,0,1
12298,User631552,Comp621781,ActiveDirectory,NetworkLogon,0,245
63018,User631552,Comp621781,ActiveDirectory,TGS,0,31
119826,User631552,Comp621781,Comp915658,TGS,0,1
142572,User631552,Comp621781,Comp915658,NetworkLogon,0,4
157549,User631552,Comp621781,ActiveDirectory,NetworkLogon,0,323
219278,User631552,Comp621781,ActiveDirectory,TGS,0,67
287722,User631552,Comp621781,Comp915658,TGS,0,3
315104,User631552,Comp621781,Comp915658,NetworkLogon,0,1


### Trying all features

In [64]:
#setwd("D://LA//ATI Data")
AU <- read.table(file="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/Advanced-Cyber-Analytics-for-Attack-Detection/Data/AuthUserNames.txt", header=F)
AU <- as.vector(AU$V1)
AU <- unique(AU)

In [65]:
DetectAnomaliesII <- function(df=Auth, s=1, q=0, m=1) {
    
   # for(i in 1:ncol(df)) {
   #     paste('t',i) <- tapply(df[,i], df[,i])
   # }
   #  t1<- tapply(df[,1], df[,1])
     t2<- tapply(df[,2], df[,2])
     t3<- tapply(df[,3], df[,3])
     t4<- tapply(df[,4], df[,4])
     t5<- tapply(df[,5], df[,5])
     t6<- tapply(df[,6], df[,6])

    dm<- as.data.frame(cbind(t2,t3,t4,t5,t6))
    data<- as.data.frame(cbind(df, samp=c(1:nrow(df))))
    dm<- as.matrix(dm)
    
    set.seed(5)
    M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(t2)), length(unique(t3)), length(unique(t4)), length(unique(t5)), length(unique(t6))))
    invisible(M)
    P1 <- cp_apr(M,2,opts=list(alg='pdnr'))
    cat("\n")
    print(P1$M)
    cat("\n")
    
    Pt1 <- as.data.frame(P1$M$u[1])
    Pt2 <- as.data.frame(P1$M$u[2])
    Pt3 <- as.data.frame(P1$M$u[3])
    p1<- s*sd(Pt1[,1]) + m*mean(Pt1[,1]) + q*IQR(Pt1[,1])
    p2<- s*sd(Pt2[,1]) + m*mean(Pt2[,1]) + q*IQR(Pt2[,1])
    p3<- s*sd(Pt3[,1]) + m*mean(Pt3[,1]) + q*IQR(Pt3[,1])

    R1<- as.numeric(row.names(Pt1[Pt1$X1>p1, ]))
    R2<- as.numeric(row.names(Pt2[Pt2$X1>p2, ]))
    R3<- as.numeric(row.names(Pt3[Pt3$X1>p3, ]))
    
    
    dm<- as.data.frame(dm)
    dmt <- dm[dm$t2%in%R1 & dm$t3%in%R2 & dm$t4%in%R3, ]
    # dmt <- dmt[dmt$t2%in%R2, ]
    # dmt <- dmt[dmt$t3%in%R3, ]
    
    print(paste("The amount of anomalous data is", nrow(dmt)/nrow(df)*100, "% of the total data provided." ))
    cat("\n")
    
    N1<- as.numeric(row.names(dmt))
    User_anom <- data[data$samp%in%N1, ]
    User_anom <- User_anom[, -ncol(User_anom)]
    print(paste("Verdict for the integrity of the process is:", identical(nrow(User_anom), length(N1))))
    cat("\n\n")
    print("The resulting dataset is:")
    cat("\n")
    print(head(User_anom, 25))
    cat("\n\n")
    
    N1<- as.vector(unique(User_anom$UserName))
    L<- N1[N1%in%AU==TRUE]
    
    print(paste(length(L)/length(N1)*100, "% of the resulting usernames feature among the list of red_team usernames."))
    
}

In [66]:
set.seed(1)
s1<- sample(1:nrow(Auth), size=nrow(Auth)*10^-4, replace=FALSE)
set.seed(2)
s2<- sample(1:nrow(Auth), size=nrow(Auth)*10^-5, replace=FALSE)
set.seed(3)
s3<- sample(1:nrow(Auth), size=nrow(Auth)*10^-3, replace=FALSE)
set.seed(4)
s4<- sample(1:nrow(Auth), size=150, replace=FALSE)
set.seed(5)
s5<- sample(1:nrow(Auth), size=nrow(Auth)/20, replace=FALSE)

A1 <- Auth[s1, ]
A2 <- Auth[s2, ]
A3 <- Auth[s3, ]
A4 <- Auth[s4, ]
A5 <- Auth[s5, ]

In [67]:
DetectAnomaliesII(A1)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 2616, KKT viol = 2.32e+00, obj = -8.70933714e+03, nz: 1136, time:1.05"
[1] "   2. Ttl Inner Its: 5597, KKT viol = 3.22e+03, obj = -5.70885808e+03, nz: 1221, time:2.89"
[1] "   3. Ttl Inner Its: 1308, KKT viol = 5.89e+00, obj = -5.70885808e+03, nz: 1221, time:3.05"
[1] "   4. Ttl Inner Its: 1349, KKT viol = 6.99e+00, obj = -5.69267046e+03, nz: 1217, time:3.27"
[1] "   5. Ttl Inner Its: 1316, KKT viol = 1.23e+00, obj = -5.69122496e+03, nz: 1218, time:3.44"
[1] "   6. Ttl Inner Its: 2278, KKT viol = 1.68e+00, obj = -5.68868507e+03, nz: 1249, time:3.99"
[1] "   7. Ttl Inner Its: 1309, KKT viol = 2.21e-02, obj = -5.68867994e+03, nz: 1249, time:4.16"
[1] "   8. Ttl Inner Its: 2263, KKT viol = 1.58e-02, obj = -5.68853563e+03, nz: 1256, time:4.66"
[1] "   9. Ttl Inner Its: 1385, KKT viol = 3.27e-03, obj = -5.68853494e+03, nz: 1256, time:4.86"
[

In [68]:
DetectAnomaliesII(A2)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 340, KKT viol = 2.89e+00, obj = -6.85269235e+02, nz: 155, time:0.16"
[1] "   2. Ttl Inner Its: 678, KKT viol = 2.98e+02, obj = -5.25137757e+02, nz: 162, time:0.36"
[1] "   3. Ttl Inner Its: 170, KKT viol = 2.85e-02, obj = -5.25137757e+02, nz: 162, time:0.38"
[1] "   4. Ttl Inner Its: 360, KKT viol = 2.61e-02, obj = -5.25112016e+02, nz: 164, time:0.47"
[1] "   5. Ttl Inner Its: 174, KKT viol = 2.98e-04, obj = -5.25112015e+02, nz: 164, time:0.50"
[1] "   6. Ttl Inner Its: 198, KKT viol = 2.77e-04, obj = -5.25112015e+02, nz: 164, time:0.52"
[1] "   7. Ttl Inner Its: 170, KKT viol = 7.71e-05, obj = -5.25112015e+02, nz: 164, time:0.55"
[1] "==========================================="
[1] " Final log-likelihood = -5.251120e+02 "
[1] " Final least squares fit = 4.856539e-02 "
[1] " Final KKT violation = 7.7140055e-05"
[1] " Total inner iterat

In [69]:
DetectAnomaliesII(A3)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 16732, KKT viol = 3.31e+00, obj = -2.49033027e+05, nz: 7040, time:8.30"
[1] "   2. Ttl Inner Its: 44041, KKT viol = 1.75e+04, obj = -5.79366342e+04, nz: 7868, time:23.80"
[1] "   3. Ttl Inner Its: 8366, KKT viol = 4.51e+01, obj = -5.79366342e+04, nz: 7868, time:24.91"
[1] "   4. Ttl Inner Its: 8705, KKT viol = 4.51e+01, obj = -5.77028516e+04, nz: 7961, time:26.02"
[1] "   5. Ttl Inner Its: 8475, KKT viol = 2.07e+01, obj = -5.76561434e+04, nz: 7962, time:27.11"
[1] "   6. Ttl Inner Its: 8614, KKT viol = 1.20e+01, obj = -5.75853553e+04, nz: 7957, time:28.33"
[1] "   7. Ttl Inner Its: 8812, KKT viol = 7.99e+00, obj = -5.75559623e+04, nz: 7959, time:29.60"
[1] "   8. Ttl Inner Its: 8663, KKT viol = 1.88e+01, obj = -5.75037833e+04, nz: 7954, time:31.10"
[1] "   9. Ttl Inner Its: 8453, KKT viol = 1.04e+01, obj = -5.74678526e+04, nz: 7949, tim

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"


[1] "==========================================="
[1] " Final log-likelihood = -5.724460e+04 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.6291622e-05"
[1] " Total inner iterations = 413523"
[1] " Total execution time = 81.00 secs"

$lambda
[1]  1079.687 11311.344

$u
$u[[1]]
                [,1]         [,2]
   [1,] 0.0000000000 8.840706e-05
   [2,] 0.0000000000 8.840706e-05
   [3,] 0.0000000000 1.768141e-04
   [4,] 0.0000000000 8.840706e-05
   [5,] 0.0000000000 1.768141e-04
   [6,] 0.0000000000 8.840706e-05
   [7,] 0.0000000000 1.768141e-04
   [8,] 0.0000000000 8.840706e-05
   [9,] 0.0000000000 8.840706e-05
  [10,] 0.0000000000 8.840706e-05
  [11,] 0.0000000000 8.840706e-05
  [12,] 0.0000000000 1.768141e-04
  [13,] 0.0000000000 8.840706e-05
  [14,] 0.0000000000 8.840706e-05
  [15,] 0.0000000000 1.768141e-04
  [16,] 0.0000000000 8.840706e-05
  [17,] 0.0000000000 8.840706e-05
  [18,] 0.0000000000 1.768141e-04
  [19,] 0.0000000000 8.840706e-05
  [20,] 0.000000000

In [70]:
DetectAnomaliesII(A4)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 398, KKT viol = 5.97e+00, obj = -8.46169734e+02, nz: 190, time:0.21"
[1] "   2. Ttl Inner Its: 822, KKT viol = 2.15e+03, obj = -6.68999058e+02, nz: 194, time:0.49"
[1] "   3. Ttl Inner Its: 199, KKT viol = 1.61e+00, obj = -6.68999058e+02, nz: 194, time:0.52"
[1] "   4. Ttl Inner Its: 280, KKT viol = 1.61e+00, obj = -6.68266415e+02, nz: 195, time:0.57"
[1] "   5. Ttl Inner Its: 211, KKT viol = 6.92e-02, obj = -6.68261536e+02, nz: 195, time:0.60"
[1] "   6. Ttl Inner Its: 365, KKT viol = 1.84e-02, obj = -6.68253953e+02, nz: 195, time:0.69"
[1] "   7. Ttl Inner Its: 227, KKT viol = 6.29e-04, obj = -6.68253951e+02, nz: 195, time:0.75"
[1] "   8. Ttl Inner Its: 224, KKT viol = 1.70e-04, obj = -6.68253951e+02, nz: 195, time:0.79"
[1] "   9. Ttl Inner Its: 205, KKT viol = 1.07e-04, obj = -6.68253951e+02, nz: 195, time:0.82"
[1] "  10. Ttl Inne

In [ ]:
# DetectAnomaliesII(A5)